In [ ]:
import pre
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
label=label=pre.create_label(1)
corpus=pre.create_sentence_list(0)

fileid=pre.fileid


In [ ]:
new_label=[]
new_tv=[]
new_fileid=[]
new_corpus=[]
length = len(corpus)
for i in range(0, length):
  for j in range(0,len(label[i])):
    new_corpus.append(corpus[i])
    new_label.append(label[i][j])
    new_fileid.append(fileid[i])
# df=pre.pd.DataFrame(pre.np.round(new_tv, 10), columns=vocab)
# #df_tfidf.insert(0, "sentences", corpus)
# df_tfidf["tag"]=new_label
# df_tfidf

In [ ]:
!nltk.download('maxent_ne_chunker')

sentences=new_corpus

In [ ]:
from nltk.tag import pos_tag
from nltk import ne_chunk
from nltk.tokenize import word_tokenize
import spacy 
from collections import Counter
nlp = spacy.load('en_core_web_sm') 

verb_count=[]
noun_count=[]
adjective_count=[]
cardinal_count=[]
is_date=[]
adverb_count=[]
for sent in sentences:
  text= pos_tag(word_tokenize(sent))
  count= Counter([j for i,j in pos_tag(word_tokenize(sent))])
  noun_count.append(count['NN']+count['NNS']+count['NNP']+count['NNPS'])
  verb_count.append(count['VBZ']+count['VB']+count['VBD']+count['VBG']+count['VBN']+count['VBP'])
  adjective_count.append(count['JJ']+count['JJR']+count['JJS'])
  cardinal_count.append(count['CD'])
  adverb_count.append(count['RB']+count['RBR']+count['RBS'])


  doc = nlp(sent)
  ner_labels= ([X.label_ for X in doc.ents])
  if 'DATE' in ner_labels:
    is_date.append(1)
  else:
    is_date.append(0)

In [ ]:
def word_count(str):
    counts = dict()
    words = str.split()

    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1

    return counts

print( word_count('the quick brown fox jumps over the lazy dog.'))

In [ ]:
mathematical_terms= ['Addition', 'Add', 'Sum', 'Plus', 'Increase', 'Total',
'Subtraction', 'Subtract', 'Minus', 'Less', 'Difference', 'Decrease', 'Deduct'
,'Multiplication', 'Multiply',' Product', 'By', 'Times','Division', 'Divide', 'Quotient','variables']

question_word=['which', 'what','whose','who', 'whom', 'whose','what', 'which','where','whither' ,'whence' ,'when','how','why','whether']

abs_terms = ['Interesting','stronger','well','written','novelty','quite','organized','clear','accept']
intro_terms=['not clear', 'missing','seems', 'quite','limited','lacking','but']
rel_wrk=['discusses', 'problem', 'present']

In [ ]:
math_count=[]
ques_wcount=[]
f1=[]
f2=[]
f3=[]


for sent in sentences:
  dic= word_count(sent)

  count=0     #mathematical
  for word in mathematical_terms:
    word=word.lower()
    if word in dic.keys():
      count=count+dic[word]
  math_count.append(count)

  count=0     #question word
  for word in question_word:
    word=word.lower()
    if word in dic.keys():
      count=count+dic[word]
  ques_wcount.append(count)

  count=0     #ABS word
  for word in abs_terms:
    word=word.lower()
    if word in dic.keys():
      count=count+dic[word]
  f1.append(count)

  count=0     #INTRO word
  for word in intro_terms:
    word=word.lower()
    if word in dic.keys():
      count=count+dic[word]
  f2.append(count)

  count=0     #REL word
  for word in rel_wrk:
    word=word.lower()
    if word in dic.keys():
      count=count+dic[word]
  f3.append(count)




In [ ]:
def punctuation(str,p):
  count = 0; 
  for i in range (0, len (str)):   
      #Checks whether given character is a punctuation mark  
      if str[i] in (p):  
          count = count + 1;  
  return count


In [ ]:
num_qmark=[]
num_excla=[]
num_per=[]
et_al=[]
for sent in sentences:
  num_qmark.append(punctuation(sent,'?'))
  num_excla.append(punctuation(sent,'!'))
  num_per.append(punctuation(sent,'%'))
  et_al.append(punctuation(sent,'et al'))


In [ ]:
avg_len=[]
for sent in sentences:
  count=0
  words= word_tokenize(sent)
  for word in words:
    count=count+len(word)
  if(len(words)!=0):
    avg_len.append(count/len(words))
  else:
    avg_len.append(0)


In [ ]:
dic={'verb':verb_count,'noun':noun_count,'adj':adjective_count,'qmark':num_qmark,'excla':num_excla,'%':num_per,'number':cardinal_count,'date':is_date,'adverb':adverb_count,\
     'math':math_count,'ques':ques_wcount,'avg_len':avg_len,'f1':f1,'f2':f2,'f3':f3}
df=pre.pd.DataFrame(dic)
df['tag']=new_label
df

CREATE X

In [ ]:
import numpy as np
X_df=df[df.columns[0:-1]] #X dataframe
X=X_df.to_numpy()    # X in array
y=df["tag"]
print(np.unique(df["tag"]))
class_names=np.unique(df["tag"])

SPLIT

In [ ]:
from sklearn.model_selection import train_test_split
from collections import Counter
X_tr, X_test, y_tr, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
y_tr=y_tr.reset_index()['tag'].tolist()
print(X_tr.shape)
print(len(y_tr))

In [ ]:
X_train=X_tr
y_train=y_tr

FITTING

In [ ]:
print(X_train.shape)
len(y_train)


LOGISTIC

In [ ]:
import sklearn.metrics as metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
logmodel = LogisticRegression(class_weight='balanced',solver='liblinear')
logmodel.fit(X_train,y_train)
y_pred = logmodel.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred,zero_division=1))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

MULTINOMIAL

In [ ]:
import sklearn.metrics as metrics
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create the model with 100 trees
clf = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt',class_weight='balanced_subsample')

import sklearn.metrics as metrics
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred,zero_division=1))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

SVM

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
# Make a decision tree and train
clf = svm.SVC(class_weight='balanced',decision_function_shape='ovo')
import sklearn.metrics as metrics
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred,zero_division=1))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

SGD

In [ ]:

from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred,zero_division=1))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

DECISION TREE

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred,zero_division=1))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))